In [1]:
from common_functions import *
import datetime
import csv
from gensim.models.ldamodel import LdaModel
import datetime
import random
import numpy as np

In [2]:
def sample_and_subset(l, n):
    if n > len(l):
        print('asked for {} elements, list length = {}'.format(n, len(l)))
    i_s = random.sample(range(len(l)), n)
    return [[l[i] for i in i_s], [l[i] for i in range(len(l)) if i not in i_s]]

In [3]:
data_dir = "Transcript_Segments"
cnn_f = data_dir + '/CNN_jan2018_{}.csv'
msnbc_f = data_dir + '/MSNBC_jan2018_{}.csv'
fox_f = data_dir + '/FOXNEWS_jan2018_{}.csv'

In [4]:
def prepare_data(labelers, N, a, condition_text, f_name):
    network_fs = [msnbc_f, cnn_f, fox_f]
    snippets_by_labeler = {labeler: [] for labeler in labelers}
    for f in network_fs:
        #f = data_dir + '/{}_full_segments.csv'.format(network)
        candidates = []
        with open(f.format('segments'),'r') as csv_f:
            for i,row in enumerate(csv.reader(line.replace('\0', ' ') for line in csv_f)):
                if i==0: continue
                if condition_text(row[7]):
                    candidates.append(row[0])
        if len(candidates) < len(labelers)*int(N*(1-a)) + int(N*a):
            print("N too big")
            return
        alpha_group, candidates = sample_and_subset(candidates, int(N*a))
        segs_by_labeler = {labeler: alpha_group for labeler in labelers}
        for labeler in labelers:
            to_label, candidates = sample_and_subset(candidates, int(N*(1-a)))
            segs_by_labeler[labeler] = segs_by_labeler[labeler] + to_label
        # now get snippets
        with open(f.format('snippets'),'r') as csv_f:
            for i,row in enumerate(csv.reader(x.replace('\0',' ') for x in csv_f)):
                if i==0: continue
                for labeler, segs in segs_by_labeler.items():
                    if row[0] in segs:
                        print(row[:2])
                        snippets_by_labeler[labeler].append((row[0], row[1], f.split('/')[-1].split('_')[0], row[8]))
    for labeler, snippets in snippets_by_labeler.items():
        rows = [["Segment", "Snippet", "Network", "text", "ABOUT TRUMP-RUSSIA?"]]
        for snippet_info in snippets:
            rows.append(list(snippet_info))
        with open(data_dir + '/labeling_data/{}_{}_unlabeled.csv'.format(labeler, f_name),'w') as csv_f:
            csv.writer(csv_f).writerows(rows)
    
    

In [6]:
def check_words(x):
    for term in ['INVESTIGATE', 'BILL', 'INTELLIGENCE', 'RUSSIA', 'PUTIN', 'HACK', 'EMAILS', 'SANCTION', 'KGB', 'MUELLER', 'COMEY', 'REPORT', 'INDICT','LEAK']:
        if term in x:
            return True
    return False

In [7]:
prepare_data(['baird'], 5, 0.0, check_words, '2018jan_data')

['10001_4', '7']
['10001_4', '8']
['10001_4', '9']
['10005_3', '7']
['10007_0', '0']
['10007_0', '1']
['10007_0', '2']
['10016_2', '6']
['10016_2', '7']
['10050_0', '0']
['10011_5', '11']
['10011_5', '12']
['10013_1', '1']
['10013_1', '2']
['10013_1', '3']
['10038_5', '10']
['10038_5', '11']
['10041_11', '14']
['10041_11', '15']
['10041_11', '16']
['10060_5', '7']
['10060_5', '8']
['10006_0', '0']
['10006_0', '1']
['10006_3', '8']
['10006_3', '9']
['10014_1', '3']
['10014_1', '4']
['10026_0', '0']
['10026_0', '1']
['10026_0', '2']
['10026_0', '3']
['10037_3', '8']
['10037_3', '9']
